In [ ]:
# graph TD
# A[리뷰 데이터 CSV] --> B[데이터 전처리] - 완
# B --> C[리뷰 → 긍/부정 라벨링(수작업)]
# C --> D[감성 분석 모델 학습]
# D --> E[새 리뷰 입력]
# E --> F[새 리뷰의 긍/부정 예측]
# F --> G[영화별 최신 리뷰 긍/부정 비율 집계]
# G --> H[사용자에게 추천/비추천 표시]

In [ ]:
# [1. 데이터 전처리]

# 1. 키노라이츠 - 트랜드 랭킹 스크레이핑
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os, csv, time

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# 메인
url = 'https://m.kinolights.com/ranking/kino'
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

detail_base_url = "https://m.kinolights.com"
movie_detail_Link = soup.select('#contents .content-list-card__body')
movie_links = []
for detail_href in movie_detail_Link:
    hrefi = detail_href.get('href')
    if hrefi:
        full_url = detail_base_url + hrefi + "?tab=review"
        movie_links.append(full_url)
print(movie_links)

# [4. 상세 정보 리뷰 페이지에서 리뷰를 전부 크롤링 ]
review_texts = []

for link in movie_links:
    print(f"크롤링 중: {link}")
    driver.get(link)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.select_one("h2").get_text(strip=True)
    reviews = soup.select('h5')

    count = 0
    for r in reviews:
        text = r.get_text(strip=True).strip()
        if text:
            review_texts.append([title, text])
            count += 1
    print(f"---------- {count} 개의 리뷰 크롤링")

# CSV 파일로 저장
folder_path = r"E:/KOSMO/TeamProject/KOSMO_AI_TP/test_data/MJ/data/"
file_path = os.path.join(folder_path, "reviews.csv")

# 폴더가 없으면 생성
os.makedirs(folder_path, exist_ok=True)

with open(file_path, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['movie_title', 'review'])  # 헤더 작성
    writer.writerows(review_texts)  # 리뷰 데이터 작성

print(f"리뷰가 '{file_path}' 파일로 {len(review_texts)}개 저장되었습니다.")
print("크롤링 완료! reviews.csv 생성")

driver.quit()

['https://m.kinolights.com/title/139423?tab=review', 'https://m.kinolights.com/title/135607?tab=review', 'https://m.kinolights.com/title/143807?tab=review', 'https://m.kinolights.com/title/126961?tab=review', 'https://m.kinolights.com/title/119041?tab=review', 'https://m.kinolights.com/title/142187?tab=review', 'https://m.kinolights.com/title/143095?tab=review', 'https://m.kinolights.com/title/130583?tab=review', 'https://m.kinolights.com/title/143536?tab=review', 'https://m.kinolights.com/title/131206?tab=review', 'https://m.kinolights.com/title/138525?tab=review', 'https://m.kinolights.com/title/70069?tab=review', 'https://m.kinolights.com/title/140653?tab=review', 'https://m.kinolights.com/title/14476?tab=review', 'https://m.kinolights.com/title/122747?tab=review', 'https://m.kinolights.com/title/102317?tab=review', 'https://m.kinolights.com/title/133383?tab=review', 'https://m.kinolights.com/title/45363?tab=review', 'https://m.kinolights.com/title/120?tab=review', 'https://m.kinoli

In [1]:
# # [2. 긍/부정 라벨링]

# 1️⃣ 사전학습된 한국어 감성 분석 모델 활용
# 예: Hugging Face nlp 허브에 올라온 snunlp/KR-FinBERT, beomi/KcELECTRA-base 등
# 이미 수십만~수백만 개 리뷰로 학습된 모델을 가져와서 → 바로 예측
# 이 모델로 1643개 리뷰를 “긍/부정” 자동 라벨링
# 2️⃣ 간단한 규칙 기반 필터 보완
# 예: “재밌다, 최고, 대박, 귀엽다, 추천” → 긍정
# “지루하다, 별로, 실망, 안 본다, 최악” → 부정
# 자동 라벨링 중 확실히 잘못된 부분만 손봐줌 (아주 적은 시간 투자)
# 3️⃣ 자동 라벨링 결과로 다시 내 모델 학습
# 이제는 내 데이터 전용 머신러닝/딥러닝 모델을 학습
# → 이후 새 리뷰 들어오면 바로 예측

In [10]:
# -----------------------------------------------------------------

In [ ]:
# [2. 데이터에 급/부정 라벨링]

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


# [1. 데이터 라벨링]

# 데이터 라벨링 함수 정의
def keyword_labeling(text):
    text = text.lower()
    pos_keywords = ['재밌', '최고', '좋아', '추천', '감동', '훌륭', '대박', '최애', '즐겁', '행복', 'ㅎ', '만족']
    neg_keywords = ['노잼', '개노잼', '별로', '싫', '지루', '쓰레기', '짜증', '끔찍', '화남', '실망', '불만', '최악',
                    '싫어', '시러', '진짜 별로', '노잼임', '못봤', '별루', '짜증나', '끔찍해', '별로야', '진짜 싫어',
                    '시무룩', '불쾌', '최악', '답답','아쉬', '촌스', '진부', '늘어짐', '늘어지', '애매', '어설',
                    '평점 알바', '아쉬운', '아쉬움']
    for kw in pos_keywords:
        if kw in text:
            return 1  # 긍정
    for kw in neg_keywords:/
        if kw in text:
            return 0  # 부정
    return -1  # 키워드가 없으면 모호

# 데이터 불러오기 및 라벨링
df = pd.read_csv('E:/KOSMO/TeamProject/KOSMO_AI_TP/test_data/MJ/data/reviews.csv')

# 자연어를 숫자로 라벨링
df['label'] = df['review'].apply(keyword_labeling)
df = df[df['label'] != -1]  # 모호한 문장 제거 (학습 데이터만)

# TF-IDF 자연어를 숫자로(벡터화) - (머신 러닝)
vectorizer = TfidfVectorizer(max_features=5000) # 상위 5000개 단어만 사용 (메모리 및 속도 고려)
X = vectorizer.fit_transform(df['review'].str.lower()) # TF-IDF로 변환된 리뷰 데이터 (행: 리뷰, 열: 단어)
y = df['label'].values # y : 레이블 배열
review_texts = df['review'].values # 원본 텍스트 보존

# -------------------------------------------------------------------------------
# [3. 학습과 테스트]

# 학습 데이터와 테스트 데이터 분리 (80% 20%)
# random_state=42 : 실행시마다 동일한 결과 재현
X_train, X_test, y_train, y_test, review_train, review_test = train_test_split(
    X, y, review_texts, test_size=0.2, random_state=42, stratify=y
)

# 로지스틱 회귀 모델 학습
# class_weight='balanced' : 부정/긍정 샘플 수가 불균형할 때 가중치를 자동으로 조절.
clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train, y_train)

# 예측
y_pred = clf.predict(X_test)

# 성능 평가 및 결과 출력
# 정확도 
accuracy = accuracy_score(y_test, y_pred)
print(f"테스트 데이터 정확도: {accuracy:.4f}")
# 클래스명 지정
print(classification_report(y_test, y_pred, target_names=['비추천(0)', '추천(1)']))

# 테스트 결과 데이터프레임 생성 및 확인
df_test = pd.DataFrame({
    'review': review_test, # 원본 리뷰
    'label': y_test, # 실제 레이블
    'predicted_label': y_pred # 예측 레이블)
})
# 각 행마다 예측이 맞았으면 '맞음', 틀렸으면 '틀림'을 'predict_data' 컬럼에 저장
df_test['predict_data'] = df_test.apply(
    lambda row: '맞음' if row['label'] == row['predicted_label'] else '틀림', axis=1
)

print(df_test.head(20))  # 상위 20개만 확인

테스트 데이터 정확도: 0.7907
              precision    recall  f1-score   support

      비추천(0)       1.00      0.25      0.40        12
       추천(1)       0.78      1.00      0.87        31

    accuracy                           0.79        43
   macro avg       0.89      0.62      0.64        43
weighted avg       0.84      0.79      0.74        43

                                               review  label  predicted_label  \
0   명품가수 임영웅의 아임 히어로 더스타디움은 볼때마다 행복하고 볼때마다 놀라움이네요 ...      1                1   
1                             현실적인 층간소음 공포영화. 추천합니다!!      1                1   
2                          열받네? 근데 현실이지...  아 근데 재밌네!      1                1   
3   정상인 없음. 공룡도 너무 멍청하고 답답함. 긴장감도 없고 에초에 죽는사람도 별로 ...      0                1   
4                          마지막 흐름전개가 대단히 아쉽지만 그래도 만족함      1                1   
5   팬들을  생각하는  마음이 진심이라는걸 다시 한번  느끼고 감동 받았어요~ 지금도 ...      1                1   
6   2부작인줄 끝에 보고 알았음.\r\n3분의2가 넘 지루한데... 뺄건 빼고 한편으로...      0                1   
7   재

In [9]:
# -------------------------------------------------------------------------------

In [ ]:
# 기존 리뷰 데이터에서 영화별 긍정/부정 리뷰 개수 집계
# 긍정 비율 계산 → 이걸로 다음 리뷰가 긍정일 확률 예측
# 기준(예: 긍정 비율 0.5 이상)으로 추천 여부 결정
# 결과를 표로 보여주기

In [ ]:
# 긍정 비율이 높으면 앞으로 긍정적인 리뷰가 많을 것으로 예측해 그 영화를 추천으로 분류

#'label' 컬럼 <= 1(긍정), 0(부정), -1(모호)
# 1. 리뷰가 모호한 리뷰도 추천
df['finall_pos_review'] = df['label'].apply(lambda x: 1 if x == -1 else x)

# 2. 영화별 긍정/부정 개수 집계 (리뷰가 없으면 제외)
# 각 영화마다 finall_pos_review=0(부정), 1(긍정)인 리뷰 수를 세고,
# .unstack(fill_value=0)으로 2개의 컬럼(0과 1)을 만듦
emotion = df.groupby(['movie_title', 'finall_pos_review']).size().unstack(fill_value=0)
emotion.columns = ['부정', '긍정'] # 칼럼명 지정(0, 1)

# 3. 영화별 모호 리뷰 개수도 따로 세기
summary_moho = df[df['label'] == -1].groupby('movie_title').size()

# 5. 긍정 비율 계산
# 긍정 비율 = 긍정 / (긍정+부정)
# 예측값 = 그냥 긍정 비율
emotion['긍정 비율'] = emotion['긍정'] / (emotion['긍정'] + emotion['부정'])
# 추천 여부 컬럼을 생성
emotion['추천 여부'] = emotion['긍정 비율'].apply(lambda x: '추천' if x >= 0.5 else '비추천')

# 5. 확인
emotion_df = emotion[['부정', '긍정', '긍정 비율', '추천 여부']]
emotion_df

,부정,긍정,긍정 비율,추천 여부
movie_title,,,,
28년 후,1,1,0.50,추천
F1 더 무비,1,3,0.75,추천
S라인,1,1,0.50,추천
견우와 선녀,0,4,1.00,추천
광장,2,2,0.50,추천
...,...,...,...,...
하트페어링,1,4,0.80,추천
핸섬가이즈,0,5,1.00,추천
화양연화,0,1,1.00,추천
